# 🧹 Data Cleaning and Preprocessing

**Project**: Cryptocurrency Market Intelligence System  
**Author**: [Your Name]  
**Course**: INSY 8413 | Introduction to Big Data Analytics  
**Date**: July 26, 2025

## 🎯 Objectives
1. Clean and validate collected cryptocurrency data
2. Handle missing values and outliers
3. Create basic derived features
4. Prepare data for advanced analysis

## 📋 Data Quality Checks
- Remove duplicates and invalid records
- Validate price data consistency (high ≥ low, close within range)
- Handle missing values appropriately
- Detect and treat outliers
- Add basic technical features

In [ ]:
# Import required libraries
import sys
import os

# Add src directory to path
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from data_processor import CryptoDataProcessor
from utils import CRYPTO_SYMBOLS, print_data_summary

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries imported successfully!")
print(f"🕐 Data cleaning started at: {datetime.now()}")

## 🔧 Initialize Data Processor

In [ ]:
# Initialize the data processor
processor = CryptoDataProcessor()

# Define our analysis parameters
SYMBOLS = ['BTC', 'ETH', 'BNB', 'ADA', 'SOL']
INTERVALS = ['5m', '1h']

print("🔧 Data processor initialized!")
print(f"📊 Symbols to process: {', '.join(SYMBOLS)}")
print(f"⏰ Intervals to process: {', '.join(INTERVALS)}")

## 📊 Data Quality Assessment

Let's first examine the quality of our raw data before cleaning.

In [ ]:
# Assess data quality for each cryptocurrency
quality_summary = []

print("🔍 ASSESSING RAW DATA QUALITY")
print("=" * 50)

for symbol in SYMBOLS:
    print(f"\n📈 {CRYPTO_SYMBOLS[symbol]} ({symbol})")
    print("-" * 30)
    
    for interval in INTERVALS:
        # Load raw data
        raw_df = processor.load_raw_data(symbol, interval)
        
        if raw_df is not None and not raw_df.empty:
            # Calculate quality metrics
            total_records = len(raw_df)
            missing_values = raw_df.isnull().sum().sum()
            duplicates = raw_df.duplicated().sum()
            
            # Check for invalid price data
            invalid_high_low = (raw_df['high'] < raw_df['low']).sum()
            invalid_close = ((raw_df['close'] > raw_df['high']) | (raw_df['close'] < raw_df['low'])).sum()
            negative_prices = (raw_df[['open', 'high', 'low', 'close']] <= 0).any(axis=1).sum()
            
            quality_metrics = {
                'symbol': symbol,
                'interval': interval,
                'total_records': total_records,
                'missing_values': missing_values,
                'duplicates': duplicates,
                'invalid_high_low': invalid_high_low,
                'invalid_close': invalid_close,
                'negative_prices': negative_prices,
                'data_quality_score': 100 - ((missing_values + duplicates + invalid_high_low + invalid_close + negative_prices) / total_records * 100)
            }
            
            quality_summary.append(quality_metrics)
            
            print(f"  {interval:3} | Records: {total_records:>6,} | Missing: {missing_values:>4} | Duplicates: {duplicates:>4} | Quality: {quality_metrics['data_quality_score']:.1f}%")
        else:
            print(f"  {interval:3} | ❌ No data found")

# Convert to DataFrame for analysis
quality_df = pd.DataFrame(quality_summary)
print(f"\n📋 Overall data quality assessment completed for {len(quality_df)} datasets")

## 🧹 Data Cleaning Process

Now let's clean all the data using our comprehensive cleaning pipeline.

In [ ]:
# Process all cryptocurrency data
print("🧹 STARTING DATA CLEANING PROCESS")
print("=" * 50)

processed_data = processor.process_all_data(SYMBOLS, INTERVALS)

print("\n✅ Data cleaning completed for all cryptocurrencies!")

## 📈 Cleaned Data Analysis

In [ ]:
# Create processing summary
summary_df = processor.create_analysis_summary(processed_data)

print("📊 CLEANED DATA SUMMARY")
print("=" * 40)
print(summary_df.to_string(index=False))

# Calculate overall statistics
total_records = summary_df['records'].sum()
avg_quality = quality_df['data_quality_score'].mean()
total_missing = summary_df['missing_values'].sum()

print(f"\n📈 OVERALL STATISTICS")
print(f"Total Records Processed: {total_records:,}")
print(f"Average Data Quality: {avg_quality:.1f}%")
print(f"Remaining Missing Values: {total_missing}")
print(f"Date Range: {summary_df['date_start'].min()} to {summary_df['date_end'].max()}")

## 📊 Data Visualization

Let's visualize the cleaned data to ensure quality.

In [ ]:
# Create comprehensive visualization of cleaned data
fig, axes = plt.subplots(3, 2, figsize=(20, 15))
fig.suptitle('Cleaned Cryptocurrency Data Analysis', fontsize=16, fontweight='bold')

# Plot 1: Price trends for hourly data
ax1 = axes[0, 0]
for i, symbol in enumerate(SYMBOLS):
    if symbol in processed_data and '1h' in processed_data[symbol]:
        df = processed_data[symbol]['1h']
        ax1.plot(df.index, df['close'], label=f'{symbol}', alpha=0.8)

ax1.set_title('Price Trends (Hourly Data)', fontweight='bold')
ax1.set_ylabel('Price (USD)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Volume analysis
ax2 = axes[0, 1]
volume_data = []
for symbol in SYMBOLS:
    if symbol in processed_data and '1h' in processed_data[symbol]:
        df = processed_data[symbol]['1h']
        volume_data.append(df['volume'].mean())
    else:
        volume_data.append(0)

bars = ax2.bar(SYMBOLS, volume_data, color=sns.color_palette("husl", len(SYMBOLS)))
ax2.set_title('Average Trading Volume', fontweight='bold')
ax2.set_ylabel('Volume')
ax2.set_yscale('log')

# Add value labels on bars
for bar, value in zip(bars, volume_data):
    if value > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                f'{value:.0e}', ha='center', va='bottom', fontweight='bold')

# Plot 3: Volatility comparison
ax3 = axes[1, 0]
volatility_data = []
for symbol in SYMBOLS:
    if symbol in processed_data and '1h' in processed_data[symbol]:
        df = processed_data[symbol]['1h']
        volatility = df['volatility_7d'].mean() * 100  # Convert to percentage
        volatility_data.append(volatility)
    else:
        volatility_data.append(0)

bars = ax3.bar(SYMBOLS, volatility_data, color=sns.color_palette("husl", len(SYMBOLS)))
ax3.set_title('Average 7-Day Volatility', fontweight='bold')
ax3.set_ylabel('Volatility (%)')

# Plot 4: Returns distribution for BTC
ax4 = axes[1, 1]
if 'BTC' in processed_data and '1h' in processed_data['BTC']:
    btc_returns = processed_data['BTC']['1h']['returns_1d'].dropna() * 100
    ax4.hist(btc_returns, bins=50, alpha=0.7, color='orange', edgecolor='black')
    ax4.axvline(btc_returns.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {btc_returns.mean():.2f}%')
    ax4.set_title('BTC Hourly Returns Distribution', fontweight='bold')
    ax4.set_xlabel('Returns (%)')
    ax4.set_ylabel('Frequency')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

# Plot 5: Data quality scores
ax5 = axes[2, 0]
quality_by_symbol = quality_df.groupby('symbol')['data_quality_score'].mean()
bars = ax5.bar(quality_by_symbol.index, quality_by_symbol.values, 
               color=sns.color_palette("husl", len(quality_by_symbol)))
ax5.set_title('Data Quality Scores by Cryptocurrency', fontweight='bold')
ax5.set_ylabel('Quality Score (%)')
ax5.set_ylim(95, 100)

# Add value labels
for bar, value in zip(bars, quality_by_symbol.values):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
             f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

# Plot 6: Record counts by interval
ax6 = axes[2, 1]
interval_counts = summary_df.groupby('interval')['records'].sum()
colors = ['skyblue', 'lightcoral']
wedges, texts, autotexts = ax6.pie(interval_counts.values, labels=interval_counts.index, 
                                   autopct='%1.1f%%', colors=colors, startangle=90)
ax6.set_title('Data Distribution by Interval', fontweight='bold')

plt.tight_layout()
plt.show()

## 🔍 Data Quality Validation

In [ ]:
# Validate cleaned data quality
print("🔍 FINAL DATA QUALITY VALIDATION")
print("=" * 50)

validation_results = []

for symbol in SYMBOLS:
    for interval in INTERVALS:
        if symbol in processed_data and interval in processed_data[symbol]:
            df = processed_data[symbol][interval]
            
            # Validation checks
            checks = {
                'symbol': symbol,
                'interval': interval,
                'total_records': len(df),
                'no_missing_critical': df[['open', 'high', 'low', 'close']].isnull().sum().sum() == 0,
                'valid_price_ranges': (df['high'] >= df['low']).all(),
                'close_in_range': ((df['close'] >= df['low']) & (df['close'] <= df['high'])).all(),
                'positive_prices': (df[['open', 'high', 'low', 'close']] > 0).all().all(),
                'sorted_chronologically': df.index.is_monotonic_increasing,
                'has_basic_features': all(col in df.columns for col in ['returns_1d', 'volatility_7d', 'close_ma_7'])
            }
            
            # Calculate overall validation score
            validation_score = sum([v for k, v in checks.items() if isinstance(v, bool)]) / 6 * 100
            checks['validation_score'] = validation_score
            
            validation_results.append(checks)
            
            # Print validation results
            status = "✅" if validation_score == 100 else "⚠️"
            print(f"{status} {symbol} {interval}: {validation_score:.0f}% validation score ({len(df):,} records)")

# Summary of validation
validation_df = pd.DataFrame(validation_results)
avg_validation_score = validation_df['validation_score'].mean()
perfect_datasets = (validation_df['validation_score'] == 100).sum()

print(f"\n📊 VALIDATION SUMMARY")
print(f"Average Validation Score: {avg_validation_score:.1f}%")
print(f"Perfect Datasets: {perfect_datasets}/{len(validation_df)}")
print(f"Total Clean Records: {validation_df['total_records'].sum():,}")

## 💾 Save Cleaning Results

In [ ]:
# Save validation and quality results
import os

# Ensure directory exists
os.makedirs('../data/processed', exist_ok=True)

# Save quality assessment
quality_df.to_csv('../data/processed/data_quality_assessment.csv', index=False)

# Save validation results
validation_df.to_csv('../data/processed/data_validation_results.csv', index=False)

# Save processing summary
summary_df.to_csv('../data/processed/processing_summary.csv', index=False)

print("💾 RESULTS SAVED")
print("=" * 20)
print("📁 data_quality_assessment.csv")
print("📁 data_validation_results.csv")
print("📁 processing_summary.csv")
print("📁 Individual processed datasets for each crypto/interval")

print("\n✅ Data cleaning and preprocessing completed successfully!")
print("➡️ Next step: Feature Engineering (03_feature_engineering.ipynb)")